In [6]:
#import libraries
import boto3
import json
import os, io
import re
import time
from io import StringIO
from io import BytesIO
from os import listdir
from os.path import isfile, join

import pytesseract

import pdfminer

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams #, LTTextBox, LTTextLine
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfparser import PDFParser

from PIL import Image

import textract

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-testing"
region = 'eu-central-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

# Checking whether the files are scanned images or true pdfs

In [ ]:
from io import StringIO
import os
s = StringIO()
s.write("abc")
pos = s.tell()
s.seek(0, os.SEEK_END)
print(s.tell())
s.seek(pos)

In [ ]:
onlyfiles = [f for f in listdir('../input/') if isfile(join('../input/', f))]

files = [f for f in onlyfiles if "~" not in f and "git" not in f]
for file in files:
    if "doc" in file:
        print("\n\n### ", file, " ###\n\n")
        try:
            print(textract.process('../input/' + file))
        except:
            print("Problemo")
            pass
        print("\n\n**************************\n\n")

In [ ]:

def is_image(file):
    fp = open(file, 'rb')
    # Create a PDF parser object associated with the file object.
    parser = PDFParser(fp)
    # Create a PDF document
    document = PDFDocument(parser)
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        return parser


In [ ]:
print(is_image('../input/India1.pdf'))